# Web Scraper for trip advisor

## Prerequisites

In [ ]:
pip install selenium

## Imports

In [2]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

## Constants

In [3]:
URL_RESTAURANTS = "https://www.tripadvisor.com/Restaurants-g189473-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html"
URL_ONE_CAFE = "https://www.tripadvisor.com/Restaurant_Review-g189473-d3807291-Reviews-To_Tsai_Thessaloniki-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html"

PATH = "chromedriver.exe"

## Custom tools

In [10]:
class element:
    COOKIES_ACCEPT_BUTTON = "onetrust-accept-btn-handler"
    SHOW_MORE_ESTABLISHMENT_TYPES_BUTTON = "fdmYH"
    RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX = 'checkbox_3'
    COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX = 'checkbox_6'

class scraper:
    def __init__(self, path, url):
        self.driver = webdriver.Chrome(PATH)
        self.url = url

    def open_browser(self):
        self.driver.get(self.url) #Load URL
        try:
            cookies = WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.ID, element.COOKIES_ACCEPT_BUTTON))
            )
            cookies.click()
        except:
            print("No cookies! Impolite! Meh...")

    def loop_at_rest(self):
        num_page = 2
        for i in range(0, num_page):
            num_items = 1
            for j in range(1, num_items+1):
                time.sleep(10)
                xpath = ".//div[@data-test='"+ str(j)+"_list_item']"
                item = self.driver.find_element(By.XPATH, xpath)
                url = item.find_element_by_xpath(".//a[@class='bHGqj Cj b']").get_attribute("href")
                self.driver.get(url)
                self.get_reviews()
#     def navigate_to_page():

    def prepare_page(self):
        # steps:
        ## We are in restaurants
        ### 1. press more
        ### 2. unclick restaurants
        ### 3. click on coffee and tea and bars.
        try:
            #there are other elements with the same class name, but this is the first one
            more_button = WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, element.SHOW_MORE_ESTABLISHMENT_TYPES_BUTTON))
            )
            more_button.click()
        except:
            pass

        #if restaurants are checked then uncheck
        try:
            restaurant_checkbox = self.driver.find_element(By.ID, element.RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX).is_selected()
            if restaurant_checkbox:
                self.driver.find_element(By.XPATH, f".//label[@for='{element.RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX}']").click()
        except:
            print("Restaurant checkbox not found!")

        #check cafes
        try:
            cafes_checkbox = self.driver.find_element(By.ID, element.COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX).is_selected()
            if not cafes_checkbox:
                self.driver.find_element(By.XPATH, f".//label[@for='{element.COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX}']").click()
        except:
            print("Cafes checkbox not found!")

        #check bars
        try:
            bars_checkbox = self.driver.find_element(By.ID, "checkbox_241").is_selected()
            if not bars_checkbox:
                self.driver.find_element(By.XPATH, ".//label[@for='checkbox_241']").click()
        except:
            print("Bars checkbox not found!")

    def get_reviews(self):
        #num_page = 10
        review_table = WebDriverWait(self.driver, 20).until(
            EC.presence_of_element_located((By.ID, "taplc_location_reviews_list_resp_rr_resp_0"))
        )
        try:
            reviews_no = self.driver.find_element(By.CLASS_NAME, "reviews_header_count").text
            reviews_no = reviews_no.replace("(", "")
            reviews_no = int(reviews_no.replace(")", ""))
            num_page = int(reviews_no / 10)
            print("page = ",num_page)
            k=0
            for i in range(0, num_page):
                # expand the review
                time.sleep(4)
                container = self.driver.find_elements(By.XPATH,".//div[@class='review-container']")
                print("container ", len(container))
                for j in range(len(container)):
                    try:
                        try:
                            time.sleep(2)
                            container[j].find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
                        except StaleElementReferenceException:
                            print("Comment ",j, " could be longer, but I will let that pass..." )

                        title = container[j].find_element_by_xpath(".//span[@class='noQuotes']").text
                        date = container[j].find_element_by_xpath(".//span[contains(@class, 'ratingDate')]").get_attribute("title")
                        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                        review = container[j].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", " ")
                        print(title)
                        print(date)
                        print(rating)
                        print(review)
                        print("-----------")
                    except NoSuchElementException:
                        print("Didn't found item ",j, ", but it should be there.")
                k=i
                # change the page
                self.driver.find_element(By.XPATH,'.//a[@class="nav next ui_button primary"]').click()
        except NoSuchElementException:
            print("Oops, unlucky search!")
        except TimeoutException:
            print("You run out of time!")
        finally:
            for i in (0,k):
                driver.execute_script("window.history.go(-1)")

    def quit_browser(self):
        self.driver.quit()


In [11]:
scraper = scraper(PATH,URL_RESTAURANTS)
scraper.open_browser()
scraper.prepare_page()
scraper.loop_at_rest()
# scraper.get_reviews()
# scraper.quit_browser()

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [ ]:
scraper.quit_browser()